# Simulation der Pendelkette

### Pendelkette (transversal)

* $N$ Pendelglieder haben Abstand $L/N$ zueinander und gehorchen der DGL:

\begin{align*}
  \frac{ d x_i(t)}{ d t} & = v_i(t) \\
  \frac{ d v_i(t)}{ d t} & = D^*/J \left( x_{i+1}(t) - 2 x_i(t) +  x_{i-1}(t) \right) - k/J ~ v_i(t)
\end{align*}

wobei für $i=1$ und $i=N$ andere Bedingungen vorgegeben werden (nur ein Nachbar).

In [ ]:
from numpy import array, concatenate, zeros
from math import sin, cos, pi, sqrt
%matplotlib notebook
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython import display
from plot import plot

### Materialparameter und Randbedingungen

In [ ]:
def AnregungLinks(t):
    #if t<.50/2: # 4 Zeilen für Anregung mit Halbpuls von 0.25s Dauer
    #    return 1*sin(2*pi/0.25*t)
    #else:
    #    return None
    return 1*sin(2*pi/2.0*t)
    
def AnregungRechts(t):
    return None # -AnregungLinks(t)
    
def Anfangsauslenkung(i):
    return 0

def Anfangsgeschwindigkeit(i):
    return 0
    
matparam =  { "N" : 48, "D*" : 10.0, "J" : 1.0, "k" : 0.1, 
             "Anregung links" : AnregungLinks,
             "Anregung rechts" : AnregungRechts,
            }
N = matparam["N"]

In [ ]:
initialvals = { "x" : zeros(N), 
                "v" : zeros(N) }
u0 = concatenate((initialvals["x"],initialvals["v"]))

for i in range(N):
    u0[i] = Anfangsauslenkung(i)
for i in range(N):
    u0[i+N] = Anfangsgeschwindigkeit(i)    

#### Störmer-Verlet Verfahren:
* Ein halber Schritt symplektischer Euler (x zuerst updaten, dann v updaten)
* Ein halber Schritt symplektischer Euler (v zuerst updaten, dann x updaten)

In [ ]:
def OneStep(u, matparam, simparam, t):
    x, v = u[0:N], u[N:2*N]
    x += 0.5 * simparam["dt"] * v
    ddx = zeros(N) # <- new array of length N with zeros
    if matparam["Anregung links"](t):
        x[0] = matparam["Anregung links"](t)
    else:
        ddx[0] = x[0] - x[1]
    if matparam["Anregung rechts"](t):
        x[N-1] = matparam["Anregung rechts"](t)
    else:
        ddx[N-1] = x[N-1] - x[N-2]
    ddx[1:N-1] = - x[2:N] - x[0:N-2] + 2*x[1:N-1]
    # ddx[N-1] = 0 # für festes Ende auf rechter Seite 
    v -= simparam["dt"] * ( matparam["D*"]/matparam["J"] * ddx + matparam["k"]/matparam["J"] * v )
    x += 0.5 * simparam["dt"] * v
    if matparam["Anregung links"](t):
        x[0] = matparam["Anregung links"](t)
    if matparam["Anregung rechts"](t):
        x[N-1] = matparam["Anregung rechts"](t)        

### Simulationsparameter

* Die Ausbreitungsgeschwindigkeit ist $c = \sqrt{D/J}$ 
* wir simulieren bis die Welle ungefähr acht mal das Intervall durchlaufen hat:

In [ ]:
c = sqrt(matparam["D*"]/matparam["J"])

NN = N
if matparam["Anregung links"]:
    NN -= 0.5
if matparam["Anregung rechts"]:
    NN -= 0.5

simparam = { "dt" : 0.5/c, "t0" : 0, "tend" : 8*NN/c, "method" : OneStep, "sampling frequency" : 1}

### Simulationslauf:

In [ ]:
t = 0
u = u0.copy()
x, v = u[0:N], u[N:2*N]
cnt = 0

positions = [u[0:N].copy()]
velocities = [u[N:2*N].copy()]

while t < simparam["tend"] + 0.5*simparam["dt"]:
    t += simparam["dt"]
    OneStep(u, matparam, simparam, t)
    if cnt % simparam["sampling frequency"] == 0:
        positions.append(u[0:N].copy())
        velocities.append(u[N:2*N].copy())    
    cnt += 1
        
print("Es wurden",len(positions),"Samples genommen.")

### Plotting

In [ ]:
fig, ax = plt.subplots(1,1)
ax.set_ylim([-1.1,1.1]);fig.set_figheight(3);fig.set_figwidth(9)
out = widgets.Output()
def on_value_change(change):
    with out:
        plot(positions[change['new']],velocities[change['new']],ax,change['new']*simparam["dt"]*simparam["sampling frequency"])
        fig.canvas.draw()   
slider = widgets.IntSlider(min=0, max=len(positions)-1, step=1, continuous_update=True)
play = widgets.Play(min=0,  max=len(positions)-1, interval=100)
slider.observe(on_value_change, 'value')
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.VBox([play, slider,out])